In [ ]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

from IPython.display import display, HTML
file_location = "G:\\Meine Ablage\\Dokumente\\Finanzen\\Banken\\N26\\n26_user_data\\n26_user_data.json"
# Open the JSON file and load it 
with open(file_location, 'r', encoding="utf8") as f:
    n26 = json.load(f)

user_id = n26["id"]
created = n26["created"]
data    = n26["data"]
categories = list(data.keys())
category_dict = {}
for category, cat_entries in data.items():
    category_dict[category] = {}
    if cat_entries==None:
        continue
    keys = list(cat_entries[0].keys())
    #category_dict[category]["keys"] = keys
    data_dict = {}
    for key in keys:
        data_dict[key] = []
    for entry in cat_entries:
        for key, value in entry.items():
            data_dict[key].append(value)
    category_dict[category]  = pd.DataFrame(data_dict)

In [ ]:
amounts = []
dates = []
comments = []
category = []

for cat in ["cash26Data", "bankTransfers", "moneyBeamData", "referralsData", "internalFeeData", "cardTransactions", "transferWiseData", "cryptoData"]:
    category += [cat]*len(category_dict[cat])

amounts += list(category_dict["cash26Data"]["amount"])
dates += list(category_dict["cash26Data"]["transaction_date"])
comments += list(category_dict["cash26Data"]["transaction_type"])

amounts += list(category_dict["bankTransfers"]["amount"])
dates += list(category_dict["bankTransfers"]["ts"])
comments += list(category_dict["bankTransfers"]["reference_text"])

amounts += list(category_dict["moneyBeamData"]["amount"])
dates += list(category_dict["moneyBeamData"]["transaction_date"])
comments += list(category_dict["moneyBeamData"]["receiver"]+": "+category_dict["moneyBeamData"]["reference_text"])

amounts += list(category_dict["referralsData"]["reward_amount"])
dates += list(category_dict["referralsData"]["invited_date"])
comments += list(category_dict["referralsData"]["contact_type"])

amounts += list(category_dict["internalFeeData"]["amount"])
dates += list(category_dict["internalFeeData"]["transaction_date"])
comments += list(category_dict["internalFeeData"]["reference_text"])

amounts += list(-category_dict["cardTransactions"]["end_amount"])
dates += list(category_dict["cardTransactions"]["transaction_date"])
card_comments = []
for name, org_amount in zip(category_dict["cardTransactions"]["merchant_name"], category_dict["cardTransactions"]["original_amount"]):
    card_comments.append(name+": "+str(org_amount))
comments += card_comments

amounts += list(-category_dict["transferWiseData"]["original_amount"]-category_dict["transferWiseData"]["fee_amount"])
dates += list(category_dict["transferWiseData"]["confirmation_timestamp"])
comments += list(category_dict["transferWiseData"]["reference_text"])

amounts += [float(amount) for amount in list(category_dict["cryptoData"]["fiat_amount"])]
dates += list(category_dict["cryptoData"]["created"])
comments += list(category_dict["cryptoData"]["asset_code"])

only_dates = []
for date_str in dates:
    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%f%z')
    except:
        try:
            date_obj = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S%z')
        except:
            date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f%z')
    only_dates.append(date_obj.date())


#amounts.append(np.nansum(amounts))
#only_dates.append(datetime.today().date())
#comments.append("Today")
#category.append("Summary")
merged_data = {"amount": amounts, "date": only_dates, "category": category,"comment": comments}
merged_df = pd.DataFrame(merged_data)
merged_df["date"] = pd.to_datetime(merged_df["date"])
merged_df = merged_df.sort_values(by='date', ascending=False)
merged_df = merged_df.set_index("date")

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', None)
display(merged_df.groupby("category").sum())
print(f"Overall Money: {merged_df['amount'].sum()}")

In [ ]:
merged_df

In [ ]:
#show individual categories
for category, df in category_dict.items():
    print(category)
    display(df)